## Plan Parsing and Conversion

This python notebook shows how to use the unified planning library to parse a plan and how to convert plans from one type to another. 

[![Open In GitHub](https://img.shields.io/badge/see-Github-579aca?logo=github)](https:///github.com/aiplan4eu/unified-planning/blob/master/docs/notebooks/12-plan-parsing-conversion.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aiplan4eu/unified-planning/blob/master/docs/notebooks/12-plan-parsing-conversion.ipynb)


### Setup

For this example there are no planners needed.

In [1]:
!pip install --pre unified-planning

Defaulting to user installation because normal site-packages is not writeable


We are now ready to use the Unified-Planning library!

### Demo


We start importing the shortcuts.

In [2]:
from unified_planning.shortcuts import *

Now we start to model a problem involving three numeric variables $c_0$, $c_1$ and $c_2$ that can be increased and decreased. The goal of this problem is to change the variables values such that  $c_0 < c_1 < c_2$. We name with value the lifted fluent that lets us access to the value of a given counter $c$.





#### Creating the fluent

First, we define the `UserTypes` and the `Fluents`.

In [3]:
Human = UserType('Human')
Supervisor = UserType('Supervisor', Human)
Worker = UserType('Worker', Human)

Location = UserType('Location')

is_supervisor_at = Fluent('is_supervisor_at', s=Supervisor, pos=Location)
is_worker_at = Fluent('is_worker_at', w=Worker, pos=Location)

is_fixed = Fluent('is_fixed', l=Location)

#### Creating the actions


In [4]:
move_supervisor = InstantaneousAction('move_supervisor', supervisor=Supervisor, l_from=Location, l_to=Location)
supervisor = move_supervisor.supervisor
l_from = move_supervisor.l_from
l_to = move_supervisor.l_to
move_supervisor.add_precondition(Not(Equals(l_from, l_to)))
move_supervisor.add_precondition(is_supervisor_at(supervisor, l_from))
move_supervisor.add_effect(is_supervisor_at(supervisor, l_from), False)
move_supervisor.add_effect(is_supervisor_at(supervisor, l_to), True)

move_worker = InstantaneousAction('move_worker', worker=Worker, l_from=Location, l_to=Location)
worker = move_worker.worker
l_from = move_worker.l_from
l_to = move_worker.l_to
move_worker.add_precondition(Not(Equals(l_from, l_to)))
move_worker.add_precondition(is_worker_at(worker, l_from))
move_worker.add_effect(is_worker_at(worker, l_from), False)
move_worker.add_effect(is_worker_at(worker, l_to), True)

fix_location = InstantaneousAction('fix_location', loc=Location, supervisor=Supervisor, worker=Worker)
loc = fix_location.loc
supervisor = fix_location.supervisor
worker = fix_location.worker
fix_location.add_precondition(is_supervisor_at(supervisor, loc))
fix_location.add_precondition(is_worker_at(worker, loc))
fix_location.add_effect(is_fixed(loc), True)


Finally, we can create a `Problem` that encompasses the fluents and the actions, and puts them together with concrete objects, an initial state and a goal. Note here that we do not need to specify all values for each object. These are set to 0 using the default intial value parameter.


In [5]:
problem = Problem('problem')

problem.add_fluent(is_supervisor_at, default_initial_value=False)
problem.add_fluent(is_worker_at, default_initial_value=False)
problem.add_fluent(is_fixed, default_initial_value=False)

s0 = Object('s0', Supervisor)
w0 = Object('w0', Worker)
l0 = Object('l0', Location)
l1 = Object('l1', Location)
l2 = Object('l2', Location)

problem.add_objects((s0, w0, l0, l1, l2))

problem.add_action(move_supervisor)
problem.add_action(move_worker)
problem.add_action(fix_location)

var_loc = Variable('l', Location)
problem.add_goal(Forall(is_fixed(var_loc), var_loc))
problem.set_initial_value(is_supervisor_at(s0, l0), True)
problem.set_initial_value(is_worker_at(w0, l0), True)



Now we see how we can generate another, larger problem, much more compactly using a more programmatic definition



In [6]:
from unified_planning.io import PDDLReader

plan_str = """(fix_location l0 s0 w0)
(move_supervisor s0 l0 l1)
(move_worker w0 l0 l1)
(fix_location l1 s0 w0)
(move_supervisor s0 l1 l2)
(move_worker w0 l1 l2)
(fix_location l2 s0 w0)
"""

reader = PDDLReader()
sequential_plan = reader.parse_plan_string(problem, plan_str)
print(sequential_plan)

SequentialPlan:
    fix_location(l0, s0, w0)
    move_supervisor(s0, l0, l1)
    move_worker(w0, l0, l1)
    fix_location(l1, s0, w0)
    move_supervisor(s0, l1, l2)
    move_worker(w0, l1, l2)
    fix_location(l2, s0, w0)


#### Solving the small and the parametric problem

The unified_planning can either select among the available planners one which is suited for the task at hand (looking at the problem kind), or use the user defined planning. In what follows we first attempt to solve the small problem with three counters and ask the UP to use a specific planning system (ENHSP), and then one with N=9 counters (problem p2) asking the UP to automatically select the engine


In [7]:
from unified_planning.plans import PlanKind

partial_order_plan = sequential_plan.convert_to(PlanKind.PARTIAL_ORDER_PLAN, problem)

print(partial_order_plan)

PartialOrderPlan:
  actions:
    0) fix_location(l0, s0, w0)
    1) move_supervisor(s0, l0, l1)
    2) move_worker(w0, l0, l1)
    3) fix_location(l1, s0, w0)
    4) move_worker(w0, l1, l2)
    5) move_supervisor(s0, l1, l2)
    6) fix_location(l2, s0, w0)
  constraints:
    0 < 1 ,2
    1 < 3
    2 < 3
    3 < 4 ,5
    5 < 6
    4 < 6


In [8]:
for i, sorted_plan in enumerate(partial_order_plan.all_sequential_plans()):
    print(i)
    print(sorted_plan)

0
SequentialPlan:
    fix_location(l0, s0, w0)
    move_worker(w0, l0, l1)
    move_supervisor(s0, l0, l1)
    fix_location(l1, s0, w0)
    move_supervisor(s0, l1, l2)
    move_worker(w0, l1, l2)
    fix_location(l2, s0, w0)
1
SequentialPlan:
    fix_location(l0, s0, w0)
    move_worker(w0, l0, l1)
    move_supervisor(s0, l0, l1)
    fix_location(l1, s0, w0)
    move_worker(w0, l1, l2)
    move_supervisor(s0, l1, l2)
    fix_location(l2, s0, w0)
2
SequentialPlan:
    fix_location(l0, s0, w0)
    move_supervisor(s0, l0, l1)
    move_worker(w0, l0, l1)
    fix_location(l1, s0, w0)
    move_supervisor(s0, l1, l2)
    move_worker(w0, l1, l2)
    fix_location(l2, s0, w0)
3
SequentialPlan:
    fix_location(l0, s0, w0)
    move_supervisor(s0, l0, l1)
    move_worker(w0, l0, l1)
    fix_location(l1, s0, w0)
    move_worker(w0, l1, l2)
    move_supervisor(s0, l1, l2)
    fix_location(l2, s0, w0)


Now let us create a problem medium-sized problem, set up a minimisation function as minimize the number of actions, and see how this can be solved optimally.